In [9]:
import pandas as pd
import numpy as np
from datetime import datetime
import random 

In [10]:
flights = pd.read_csv('flights.csv')
airlines = pd.read_csv('airlines.csv')
airports = pd.read_csv('airports.csv')

C:\Users\Wendy\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


This notebook used '2015 Flight Delays and Cancellations' data at Kaggle: https://www.kaggle.com/usdot/flight-delays. 

There are three sources under this Kaggle dataset:
   1. **flights** contains all US domestic flights in 2015. 

    Note: 
    1. the **flights** dataset is very large, with 5,819,079 records. For the effeciency of processing in Python and in Tableau, I randomly sampled 10,000 records and performed data cleaning.
    2. the **flights** dataset has inconsistency issue while encoding airports. In October, the airports are encoded in 5-digit numbers (12892) rather than 3-digit letters (ABE). Here I used this kernel by Scott Cole https://www.kaggle.com/srcole/fix-inconsistent-airport-codes to fix the inconsistency issue.  
      
   
   2. **airlines** contains airline IATA code (e.g. UA) and corresponding airline names(e.g. United Airlines Inc.). 
   3. **airports** contains airport IATA code (e.g. ABE) and corresponding airport geographical information. 

## Flight Dataset

### Flight Note 1, sample 10,000 records and clean.

In [103]:
random.seed(72)

In [104]:
df_f1 = pd.DataFrame(flights.sample(10000))

In [105]:
df_f1.head().transpose()

,2532806,2325680,1474881,2773425,2558091
YEAR,2015,2015,2015,2015,2015
MONTH,6,5,4,6,6
DAY,9,28,5,24,11
DAY_OF_WEEK,2,4,7,3,4
AIRLINE,US,HA,DL,WN,WN
FLIGHT_NUMBER,851,121,2110,4031,572
TAIL_NUMBER,N184US,N477HA,N988DL,N209WN,N627SW
ORIGIN_AIRPORT,PHL,ITO,DFW,OAK,BNA
DESTINATION_AIRPORT,DEN,HNL,ATL,SAN,DAL
SCHEDULED_DEPARTURE,1845,849,1315,620,955


Check the completeness of the sampled dataset.

In [106]:
(len(df_f1) - df_f1.isna().sum())/len(df_f1)*100

YEAR                   100.00
MONTH                  100.00
DAY                    100.00
DAY_OF_WEEK            100.00
AIRLINE                100.00
FLIGHT_NUMBER          100.00
TAIL_NUMBER             99.75
ORIGIN_AIRPORT         100.00
DESTINATION_AIRPORT    100.00
SCHEDULED_DEPARTURE    100.00
DEPARTURE_TIME          98.21
DEPARTURE_DELAY         98.21
TAXI_OUT                98.16
WHEELS_OFF              98.16
SCHEDULED_TIME         100.00
ELAPSED_TIME            97.92
AIR_TIME                97.92
DISTANCE               100.00
WHEELS_ON               98.11
TAXI_IN                 98.11
SCHEDULED_ARRIVAL      100.00
ARRIVAL_TIME            98.11
ARRIVAL_DELAY           97.92
DIVERTED               100.00
CANCELLED              100.00
CANCELLATION_REASON      1.86
AIR_SYSTEM_DELAY        18.30
SECURITY_DELAY          18.30
AIRLINE_DELAY           18.30
LATE_AIRCRAFT_DELAY     18.30
WEATHER_DELAY           18.30
dtype: float64

In [117]:
##Drop Tail_Number which we do not need.
df_f1 = df_f1.drop(columns = ['TAIL_NUMBER'])

Why there are so many missing values in *CANCELLATION_REASON* and 5 delay reasons? Thanks to the kernel by DSEverything https://www.kaggle.com/dongxu027/airline-delays-eda-deep-dive-lessons-learned/notebook, only cancelled flights have cancelation reasons. Only flights that arrived late for more than 15 minutes would be treated as delayed flights and have delay reasons.

In [118]:
delayed_flights = df_f1[(df_f1['ARRIVAL_DELAY'] >= 15)]

In [119]:
len(delayed_flights)/10000

0.183

0.183 matches with the completion rate of delay reasons (*AIR_SYSTEM_DELAY*, *SECURITY_DELAY*, *AIRLINE_DELAY*, *LATE_AIRCRAFT_DELAY* and *WEATHER_DELAY*. Confirms that only delayed flights have delay reason.

In [120]:
cancelled_flights = df_f1[df_f1['CANCELLED'] == 1]

In [121]:
len(cancelled_flights)/10000

0.0186

0.0186 matches with the completion rate of cancellation reasons. Comfirms that only cancelled flights have cancellation reason.

The missing values in other columns might be a result of cancelled flights too. If a flight is cancelled, then it should not have any departure time/ arrival time. Some flights even made to the runway and even took off (UA249 at 2015/11/20 and WN4568 at 2015/6/1) but got cancelled shortly. The departure time and departure delay does not really have any meaning and might affact the delay time calculation later on. Therefore, I replaced the value of *DEPARTURE_TIME*, *DEPARTURE_DELAY*, *TAXI_OUT* and *WHEELS_OFF* with NA.

In [128]:
cancelled_flights.isna().sum()

YEAR                     0
MONTH                    0
DAY                      0
DAY_OF_WEEK              0
AIRLINE                  0
FLIGHT_NUMBER            0
ORIGIN_AIRPORT           0
DESTINATION_AIRPORT      0
SCHEDULED_DEPARTURE      0
DEPARTURE_TIME         179
DEPARTURE_DELAY        179
TAXI_OUT               184
WHEELS_OFF             184
SCHEDULED_TIME           0
ELAPSED_TIME           186
AIR_TIME               186
DISTANCE                 0
WHEELS_ON              186
TAXI_IN                186
SCHEDULED_ARRIVAL        0
ARRIVAL_TIME           186
ARRIVAL_DELAY          186
DIVERTED                 0
CANCELLED                0
CANCELLATION_REASON      0
AIR_SYSTEM_DELAY       186
SECURITY_DELAY         186
AIRLINE_DELAY          186
LATE_AIRCRAFT_DELAY    186
WEATHER_DELAY          186
dtype: int64

In [133]:
cancelled_flights[cancelled_flights['WHEELS_OFF'].notnull()].transpose()

,5189991,2399226
YEAR,2015,2015
MONTH,11,6
DAY,20,1
DAY_OF_WEEK,5,1
AIRLINE,UA,WN
FLIGHT_NUMBER,249,4568
ORIGIN_AIRPORT,IAH,PBI
DESTINATION_AIRPORT,MFE,BWI
SCHEDULED_DEPARTURE,2135,1825
DEPARTURE_TIME,2208,1822


(feel sorry for passengers of WN4568 at 2015/6/1, sat in the plane on the runway for 107 minutes and then got cancelled due to weather:(

In [140]:
cancelled_flights[cancelled_flights['DEPARTURE_DELAY'].notnull()]

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
2057355,2015,5,11,1,EV,4403,CMH,IAH,1441,NaN,...,NaN,NaN,0,1,B,NaN,NaN,NaN,NaN,NaN
2285743,2015,5,25,1,EV,4367,IAH,MCI,1915,NaN,...,NaN,NaN,0,1,B,NaN,NaN,NaN,NaN,NaN
5189991,2015,11,20,5,UA,249,IAH,MFE,2135,NaN,...,NaN,NaN,0,1,B,NaN,NaN,NaN,NaN,NaN
1270577,2015,3,23,1,MQ,3438,DFW,CVG,1950,NaN,...,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN
5750617,2015,12,27,7,EV,4540,IAH,PNS,1535,NaN,...,NaN,NaN,0,1,B,NaN,NaN,NaN,NaN,NaN
2399226,2015,6,1,1,WN,4568,PBI,BWI,1825,NaN,...,NaN,NaN,0,1,B,NaN,NaN,NaN,NaN,NaN
882925,2015,2,27,5,MQ,3090,DFW,LCH,1805,NaN,...,NaN,NaN,0,1,B,NaN,NaN,NaN,NaN,NaN


In [141]:
cancelled_flights.loc[2057355,'DEPARTURE_TIME'] = None
cancelled_flights.loc[2285743,'DEPARTURE_TIME'] = None
cancelled_flights.loc[5189991,'DEPARTURE_TIME'] = None
cancelled_flights.loc[1270577,'DEPARTURE_TIME'] = None
cancelled_flights.loc[5750617,'DEPARTURE_TIME'] = None
cancelled_flights.loc[2399226,'DEPARTURE_TIME'] = None
cancelled_flights.loc[882925,'DEPARTURE_TIME'] = None
cancelled_flights.loc[2057355,'DEPARTURE_DELAY'] = None
cancelled_flights.loc[2285743,'DEPARTURE_DELAY'] = None
cancelled_flights.loc[5189991,'DEPARTURE_DELAY'] = None
cancelled_flights.loc[1270577,'DEPARTURE_DELAY'] = None
cancelled_flights.loc[5750617,'DEPARTURE_DELAY'] = None
cancelled_flights.loc[2399226,'DEPARTURE_DELAY'] = None
cancelled_flights.loc[882925,'DEPARTURE_DELAY'] = None
cancelled_flights.loc[5189991,'TAXI_OUT'] = None
cancelled_flights.loc[5189991,'WHEELS_OFF'] = None
cancelled_flights.loc[2399226,'TAXI_OUT'] = None
cancelled_flights.loc[2399226,'WHEELS_OFF'] = None

In [143]:
cancelled_flights.isna().sum()

YEAR                     0
MONTH                    0
DAY                      0
DAY_OF_WEEK              0
AIRLINE                  0
FLIGHT_NUMBER            0
ORIGIN_AIRPORT           0
DESTINATION_AIRPORT      0
SCHEDULED_DEPARTURE      0
DEPARTURE_TIME         186
DEPARTURE_DELAY        186
TAXI_OUT               186
WHEELS_OFF             186
SCHEDULED_TIME           0
ELAPSED_TIME           186
AIR_TIME               186
DISTANCE                 0
WHEELS_ON              186
TAXI_IN                186
SCHEDULED_ARRIVAL        0
ARRIVAL_TIME           186
ARRIVAL_DELAY          186
DIVERTED                 0
CANCELLED                0
CANCELLATION_REASON      0
AIR_SYSTEM_DELAY       186
SECURITY_DELAY         186
AIRLINE_DELAY          186
LATE_AIRCRAFT_DELAY    186
WEATHER_DELAY          186
dtype: int64

Now take a look of not cancelled flights. Remove the records if there is NA in any column other than delay reason/ cancel reason.

In [144]:
Not_cancelled_flights = df_f1[df_f1['CANCELLED'] == 0]

In [145]:
Not_cancelled_flights.isna().sum()

YEAR                      0
MONTH                     0
DAY                       0
DAY_OF_WEEK               0
AIRLINE                   0
FLIGHT_NUMBER             0
ORIGIN_AIRPORT            0
DESTINATION_AIRPORT       0
SCHEDULED_DEPARTURE       0
DEPARTURE_TIME            0
DEPARTURE_DELAY           0
TAXI_OUT                  0
WHEELS_OFF                0
SCHEDULED_TIME            0
ELAPSED_TIME             22
AIR_TIME                 22
DISTANCE                  0
WHEELS_ON                 3
TAXI_IN                   3
SCHEDULED_ARRIVAL         0
ARRIVAL_TIME              3
ARRIVAL_DELAY            22
DIVERTED                  0
CANCELLED                 0
CANCELLATION_REASON    9814
AIR_SYSTEM_DELAY       7984
SECURITY_DELAY         7984
AIRLINE_DELAY          7984
LATE_AIRCRAFT_DELAY    7984
WEATHER_DELAY          7984
dtype: int64

In [148]:
Not_cancelled_flights2 = Not_cancelled_flights.dropna(subset = ['DEPARTURE_TIME','DEPARTURE_DELAY',\
                                                                 'TAXI_OUT','WHEELS_OFF', 'ELAPSED_TIME','AIR_TIME',\
                                                                 'WHEELS_ON','TAXI_IN','ARRIVAL_TIME','ARRIVAL_DELAY'])

In [149]:
Not_cancelled_flights2.isna().sum()

YEAR                      0
MONTH                     0
DAY                       0
DAY_OF_WEEK               0
AIRLINE                   0
FLIGHT_NUMBER             0
ORIGIN_AIRPORT            0
DESTINATION_AIRPORT       0
SCHEDULED_DEPARTURE       0
DEPARTURE_TIME            0
DEPARTURE_DELAY           0
TAXI_OUT                  0
WHEELS_OFF                0
SCHEDULED_TIME            0
ELAPSED_TIME              0
AIR_TIME                  0
DISTANCE                  0
WHEELS_ON                 0
TAXI_IN                   0
SCHEDULED_ARRIVAL         0
ARRIVAL_TIME              0
ARRIVAL_DELAY             0
DIVERTED                  0
CANCELLED                 0
CANCELLATION_REASON    9792
AIR_SYSTEM_DELAY       7962
SECURITY_DELAY         7962
AIRLINE_DELAY          7962
LATE_AIRCRAFT_DELAY    7962
WEATHER_DELAY          7962
dtype: int64

Finally, union cancelled and not cancelled fligts.

In [150]:
df_f1_cleaned = pd.concat([Not_cancelled_flights2,cancelled_flights],ignore_index=True )

In [151]:
df_f1_cleaned.shape

(9978, 30)

### Flight Note 2, Fix the inconsistancy of airport codes.

In [110]:
##A quick look of the inconsistancy code problem. 
##Some airports are encoded in 'ANC' or 'LAX', but others are encoded in '14717'.
flights['ORIGIN_AIRPORT'].unique()

array(['ANC', 'LAX', 'SFO', 'SEA', 'LAS', 'DEN', 'SLC', 'PDX', 'FAI',
       'MSP', 'PHX', 'SJU', 'PBG', 'IAG', 'PSE', 'BQN', 'ORD', 'GEG',
       'HNL', 'ONT', 'MCO', 'BOS', 'HIB', 'ABR', 'MAF', 'DFW', 'MKE',
       'IAH', 'BNA', 'BRO', 'VPS', 'BOI', 'BJI', 'SGF', 'PHL', 'SBN',
       'RDD', 'EUG', 'IAD', 'BUF', 'PWM', 'JFK', 'CRP', 'PIA', 'FAT',
       'SMF', 'AUS', 'MCI', 'ATL', 'JAX', 'MFR', 'IDA', 'MSN', 'DCA',
       'SAT', 'CHS', 'SBA', 'SMX', 'IND', 'CLE', 'GSP', 'BDL', 'ABI',
       'RIC', 'BFL', 'OMA', 'RDM', 'FLL', 'CID', 'TPA', 'SYR', 'ROC',
       'TYR', 'LAN', 'XNA', 'GSO', 'EWR', 'PBI', 'RSW', 'OAK', 'PVD',
       'RNO', 'PIT', 'ABQ', 'MIA', 'BWI', 'LGA', 'TUL', 'LIT', 'MSY',
       'OKC', 'ATW', 'PNS', 'MEM', 'TYS', 'MHT', 'SAV', 'CLT', 'GRB',
       'ABE', 'JAN', 'OAJ', 'FAR', 'ERI', 'LEX', 'CWA', 'MSO', 'TTN',
       'AMA', 'CLL', 'HOU', 'JLN', 'MLI', 'RDU', 'CVG', 'MHK', 'MOB',
       'TLH', 'BHM', 'CAE', 'TXK', 'ACY', 'DTW', 'RAP', 'TUS', 'EAU',
       'DLH', 'FSD',

Now I am fixing this inconsistency issue using the method from Scott Cole's kernel. Basically, what the code does is to match 5-digit codes with 3-digit codes, then replace 5-digit codes with 3-digit codes.   
  
This process need two extra files: **L_AIRPORT.csv** and **L_AIRPORT_ID.csv**. The first one is the 3-digit codes and the second is the 5-digit codes. The original data of those two files is listed on the website of Bureau of Transportation Statistics.   
  
The official 5-digit code from Bureau of Transportation Statistics is here:  
https://www.transtats.bts.gov/FieldInfo.asp?Field_Desc=Origin%20Airport%2C%20Airport%20ID.%20An%20identification%20number%20assigned%20by%20US%20DOT%20to%20identify%20a%20unique%20airport.%20%20Use%20this%20field%20for%20airport%20analysis%20across%20a%20range%20of%20years%20because%20an%20airport%20can%20change%20its%20airport%20code%20and%20airport%20codes%20can%20be%20reused.&Field_Type=Num&Lookup_Table=L_AIRPORT_ID&Table_ID=293&SYS_Table_Name=T_T100_SEGMENT_ALL_CARRIER&Sys_Field_Name=ORIGIN_AIRPORT_ID
The official 3-digit code from Bureau of Transportation Statistics is here:  
https://www.bts.gov/topics/airlines-and-airports/world-airport-codes   
  
Note: The fixing process is very time-consuming. It' better to just perform the process on the cleaned sample dataset than the whole dataset (which will take about 100+ hours to perform in my computer:(

In [152]:
##Read in 3-digit code and 5-digit code file
df_aircode1 = pd.read_csv('L_AIRPORT.csv')
df_aircode2 = pd.read_csv('L_AIRPORT_ID.csv')

In [153]:
##Create a dictionary to match 3-digit code and 5-digit code
df_aircode1 = df_aircode1.reset_index()
df_aircode2 = df_aircode2.reset_index()
df_aircodes = pd.merge(df_aircode1,df_aircode2,on='Description')
aircode_dict = dict(zip(df_aircodes['Code_y'].astype(str),df_aircodes['Code_x']))

In [154]:
aircode_dict

{'10001': '01A',
 '10003': '03A',
 '10004': '04A',
 '10005': '05A',
 '10006': '06A',
 '10007': '07A',
 '10008': '08A',
 '10009': '09A',
 '10010': '1B1',
 '10011': 'DQR',
 '11398': 'DQR',
 '10012': '1N7',
 '16571': '1NY',
 '16429': '6B0',
 '16567': '7AK',
 '10013': '8F3',
 '10014': 'A01',
 '10015': 'A02',
 '10016': 'A03',
 '10017': 'A04',
 '10018': 'A05',
 '10019': 'A06',
 '10020': 'A07',
 '10021': 'A08',
 '10022': 'A09',
 '10024': 'A11',
 '10025': 'A12',
 '10026': 'A13',
 '10027': 'A14',
 '10028': 'A15',
 '10029': 'A16',
 '10030': 'A17',
 '10031': 'A18',
 '16668': 'A1K',
 '10033': 'DQH',
 '11388': 'DQH',
 '10034': 'A21',
 '10035': 'A22',
 '10036': 'A23',
 '10037': 'A24',
 '10038': 'A25',
 '10039': 'A26',
 '10040': 'A27',
 '10041': 'A28',
 '10042': 'A29',
 '10770': 'BSZ',
 '10043': 'A30',
 '10044': 'A31',
 '10047': 'A34',
 '11828': 'A35',
 '10049': 'A36',
 '10050': 'A37',
 '10051': 'A38',
 '10052': 'A39',
 '16696': 'A3K',
 '10053': 'A40',
 '10054': 'A41',
 '10055': 'A42',
 '10056': 'A43

In [155]:
# Make sure all Origin and departing airports are strings
df_f1_cleaned['ORIGIN_AIRPORT'] = df_f1_cleaned['ORIGIN_AIRPORT'].values.astype(str)
df_f1_cleaned['DESTINATION_AIRPORT'] = df_f1_cleaned['DESTINATION_AIRPORT'].values.astype(str)

In [156]:
# Replace 3-digit with 5 digit
N_flights = len(df_f1_cleaned)
for i in range(N_flights):
    if i % 100 == 0:
        print(i)
    if len(df_f1_cleaned['ORIGIN_AIRPORT'][i]) != 3:
        to_replace = df_f1_cleaned['ORIGIN_AIRPORT'][i]
        value = aircode_dict[df_f1_cleaned['ORIGIN_AIRPORT'][i]]
        df_f1_cleaned = df_f1_cleaned.replace(to_replace, value)
        print('replaced',to_replace,'with',value)
    elif len(df_f1_cleaned['DESTINATION_AIRPORT'][i]) != 3:
        to_replace = df_f1_cleaned['DESTINATION_AIRPORT'][i]
        value = aircode_dict[df_f1_cleaned['DESTINATION_AIRPORT'][i]]
        df_f1_cleaned = df_f1_cleaned.replace(to_replace, value)
        print('replaced',to_replace,'with',value)

0
replaced 15380 with TVC
replaced 15304 with TPA
replaced 14843 with SJU
replaced 11298 with DFW
replaced 15370 with TUL
replaced 14771 with SFO
replaced 13930 with ORD
replaced 13232 with MDW
replaced 11618 with EWR
100
replaced 10693 with BNA
replaced 11292 with DEN
replaced 12953 with LGA
replaced 13204 with MCO
replaced 14100 with PHL
replaced 12173 with HNL
replaced 11697 with FLL
replaced 12478 with JFK
200
replaced 12448 with JAN
replaced 11617 with EWN
replaced 13487 with MSP
replaced 14831 with SJC
replaced 11921 with GJT
replaced 12266 with IAH
replaced 13342 with MKE
replaced 11982 with GRK
replaced 11193 with CVG
replaced 12441 with JAC
300
replaced 10721 with BOS
replaced 13931 with ORF
replaced 12889 with LAS
replaced 14027 with PBI
replaced 14685 with SAV
replaced 14635 with RSW
replaced 11109 with COS
replaced 11433 with DTW
400
replaced 10423 with BSM
replaced 13851 with OKC
replaced 12892 with LAX
replaced 14107 with PHX
replaced 13198 with MCI
replaced 13303 with MI

In [167]:
df_f1_cleaned['ORIGIN_AIRPORT'].unique()

array(['PHL', 'ITO', 'DFW', 'OAK', 'BNA', 'MSP', 'TVC', 'ORD', 'LAX',
       'MCO', 'ATL', 'ISN', 'SLC', 'DTW', 'LAS', 'PHX', 'TPA', 'FAT',
       'SEA', 'SRQ', 'JFK', 'BOS', 'JAX', 'MIA', 'HNL', 'DEN', 'LGA',
       'ABQ', 'SJU', 'SFO', 'MDW', 'BIS', 'IAH', 'ROA', 'RDU', 'CLT',
       'SMF', 'FLL', 'EWR', 'PSP', 'STL', 'CRP', 'TUL', 'SJC', 'SDF',
       'IAD', 'PDX', 'DCA', 'IND', 'CVG', 'PBI', 'SAN', 'PWM', 'LBE',
       'CMH', 'ANC', 'BWI', 'MEM', 'LGB', 'OKC', 'DSM', 'PIT', 'RIC',
       'SNA', 'MSY', 'ILM', 'ELP', 'HOU', 'GSP', 'ALB', 'PVD', 'EWN',
       'CID', 'HPN', 'AUS', 'LIH', 'RAP', 'OMA', 'FNT', 'GJT', 'MKE',
       'MYR', 'CHS', 'GRK', 'BDL', 'MLU', 'JAC', 'OGG', 'DAL', 'PIH',
       'LFT', 'TLH', 'CAE', 'GCC', 'BRW', 'GEG', 'GRB', 'MHT', 'RSW',
       'KTN', 'COS', 'SYR', 'FCA', 'BUR', 'MRY', 'TRI', 'DRO', 'PIB',
       'MCI', 'ECP', 'MDT', 'SAT', 'TYS', 'SGF', 'LWS', 'CLE', 'SBN',
       'ATW', 'ROC', 'TUS', 'LIT', 'LBB', 'FSD', 'JAN', 'BOI', 'EVV',
       'BTR', 'GNV',

In [162]:
df_f1_cleaned['DESTINATION_AIRPORT'].unique()

array(['DEN', 'HNL', 'ATL', 'SAN', 'DAL', 'ORD', 'LAX', 'MSP', 'CAE',
       'PDX', 'BDL', 'TPA', 'JFK', 'DCA', 'STL', 'EWR', 'RNO', 'SEA',
       'CMI', 'HLN', 'HPN', 'OKC', 'PHX', 'MIA', 'DIK', 'OGG', 'PHL',
       'MYR', 'CLE', 'LIH', 'MLU', '15249', 'YUM', 'AUS', 'LGA', 'JAX',
       'DFW', 'SPI', 'BOS', 'BUR', 'IAH', 'GEG', 'HOU', 'SFO', 'MSY',
       'MEM', 'IAD', 'LAN', 'PIT', 'ANC', 'DTW', 'MCO', 'ABQ', 'SNA',
       'SJT', 'TUS', 'RIC', 'SLC', 'BNA', 'ONT', 'SCE', 'BTV', 'SJC',
       'PVD', 'MCI', 'BWI', '12391', 'TRI', 'LAS', 'MDW', 'PIA', 'CLT',
       'ABY', 'ITO', 'OMA', 'IND', 'FAI', 'GPT', 'FAR', 'PSP', 'SGF',
       'JAN', 'BTR', 'DLH', 'CRP', 'SAT', 'SJU', 'MKE', 'DSM', 'CRW',
       'FLL', 'TVC', 'PHF', 'RAP', 'FLG', 'ACV', 'AEX', 'LGB', 'EYW',
       'OAK', 'CMH', 'ICT', 'GRR', 'ORF', 'ALB', 'SMF', 'ELP', 'FWA',
       'SAV', 'FAT', 'LIT', 'GRB', 'RSW', 'BSM', 'MEI', 'FSM', 'SBN',
       'KOA', 'TUL', 'COU', 'RDU', 'BUF', 'TYS', 'SMX', 'GRK', 'GNV',
       'COS', 'I

Two records are not converted successfully. Here I manually replaced the codes.

In [163]:
df_f1_cleaned[df_f1_cleaned['DESTINATION_AIRPORT'] == '15249']

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
48,2015,10,7,3,EV,2818,DFW,15249,1835,1833.0,...,2130.0,-18.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [169]:
df_f1_cleaned.loc[48,'DESTINATION_AIRPORT'] = 'TLH'

In [165]:
df_f1_cleaned[df_f1_cleaned['DESTINATION_AIRPORT'] == '12391']

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
131,2015,10,8,4,WN,2159,MCO,12391,2045,2043.0,...,2305.0,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [170]:
df_f1_cleaned.loc[131,'DESTINATION_AIRPORT'] = 'ISP'

In [171]:
df_f1_cleaned['DESTINATION_AIRPORT'].unique()

array(['DEN', 'HNL', 'ATL', 'SAN', 'DAL', 'ORD', 'LAX', 'MSP', 'CAE',
       'PDX', 'BDL', 'TPA', 'JFK', 'DCA', 'STL', 'EWR', 'RNO', 'SEA',
       'CMI', 'HLN', 'HPN', 'OKC', 'PHX', 'MIA', 'DIK', 'OGG', 'PHL',
       'MYR', 'CLE', 'LIH', 'MLU', 'TLH', 'YUM', 'AUS', 'LGA', 'JAX',
       'DFW', 'SPI', 'BOS', 'BUR', 'IAH', 'GEG', 'HOU', 'SFO', 'MSY',
       'MEM', 'IAD', 'LAN', 'PIT', 'ANC', 'DTW', 'MCO', 'ABQ', 'SNA',
       'SJT', 'TUS', 'RIC', 'SLC', 'BNA', 'ONT', 'SCE', 'BTV', 'SJC',
       'PVD', 'MCI', 'BWI', 'ISP', 'TRI', 'LAS', 'MDW', 'PIA', 'CLT',
       'ABY', 'ITO', 'OMA', 'IND', 'FAI', 'GPT', 'FAR', 'PSP', 'SGF',
       'JAN', 'BTR', 'DLH', 'CRP', 'SAT', 'SJU', 'MKE', 'DSM', 'CRW',
       'FLL', 'TVC', 'PHF', 'RAP', 'FLG', 'ACV', 'AEX', 'LGB', 'EYW',
       'OAK', 'CMH', 'ICT', 'GRR', 'ORF', 'ALB', 'SMF', 'ELP', 'FWA',
       'SAV', 'FAT', 'LIT', 'GRB', 'RSW', 'BSM', 'MEI', 'FSM', 'SBN',
       'KOA', 'TUL', 'COU', 'RDU', 'BUF', 'TYS', 'SMX', 'GRK', 'GNV',
       'COS', 'ISN',

Problem fixed!

The final step, concat *YEAR*, *MONTH*,*DAY* columns into one *date* column, add a *FLIGHT_ID* column as the unique identifier and export the dataset.

In [199]:
df_f1_cleaned['DATE'] = pd.to_datetime(df_f1_cleaned[['YEAR','MONTH', 'DAY']])

In [200]:
df_f1_cleaned.head(1)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,...,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE
0,2015,6,9,2,US,851,PHL,DEN,1845,1838.0,...,8.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-06-09


In [201]:
df_f1_cleaned['FLIGHT_ID'] = range(len(df_f1_cleaned))

In [202]:
df_f1_cleaned.head(1)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,...,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE,FLIGHT_ID
0,2015,6,9,2,US,851,PHL,DEN,1845,1838.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-06-09,0


In [203]:
df_f1_cleaned.to_csv('flights_sampled_cleaned.csv',index = False)

In [205]:
df_f1_cleaned.shape

(9978, 32)

## Airlines Dataset

**airlines** dataset contains airline information of 14 USA airline companies. This dataset maps airline IATA code (e.g. UA) to airline name (e.g. United Airlines). This dataset is complete. This dataset can be joined to **flights** dataset in Tableau.

In [191]:
airlines.isna().sum()

IATA_CODE    0
AIRLINE      0
dtype: int64

In [192]:
airlines.shape

(14, 2)

## Airports Dataset

**airports** dataset contains airport information of 322 USA airports. This dataset maps IATA code (e.g. ABE) to airport name, city, state, country, latitude and longitude. This dataset is not complete. 3 airports do not have longitude and latitude. Luckily, these missing values can be filled using Google Maps.

In [193]:
airports.shape

(322, 7)

In [194]:
airports.isna().sum()

IATA_CODE    0
AIRPORT      0
CITY         0
STATE        0
COUNTRY      0
LATITUDE     3
LONGITUDE    3
dtype: int64

In [195]:
airports.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [196]:
airports[airports['LATITUDE'].isnull()]

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
96,ECP,Northwest Florida Beaches International Airport,Panama City,FL,USA,NaN,NaN
234,PBG,Plattsburgh International Airport,Plattsburgh,NY,USA,NaN,NaN
313,UST,Northeast Florida Regional Airport (St. August...,St. Augustine,FL,USA,NaN,NaN


In [197]:
airports.loc[airports['IATA_CODE']== 'ECP', 'LATITUDE'] = 30.3553
airports.loc[airports['IATA_CODE']== 'ECP', 'LONGITUDE'] = 85.7991
airports.loc[airports['IATA_CODE']== 'PBG', 'LATITUDE'] = 44.6519
airports.loc[airports['IATA_CODE']== 'PBG', 'LONGITUDE'] = 73.4657
airports.loc[airports['IATA_CODE']== 'UST', 'LATITUDE'] = 29.9544
airports.loc[airports['IATA_CODE']== 'UST', 'LONGITUDE'] = 81.3429

In [198]:
airports.isna().sum()

IATA_CODE    0
AIRPORT      0
CITY         0
STATE        0
COUNTRY      0
LATITUDE     0
LONGITUDE    0
dtype: int64

In [204]:
airports.to_csv('airports_cleaned.csv', index= False)